In [1]:
# Simple CNN model
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn import preprocessing

import keras
K.set_image_dim_ordering('th')


Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [12]:
# load data
X_train = np.load(open('train_dataset.npy','rb'))
y_train = np.load(open('train_labels.npy','rb'))
X_test = np.load(open('test_dataset.npy','rb'))
y_test = np.load(open('test_labels.npy','rb'))
X_valid = np.load(open('valid_dataset.npy','rb'))
y_valid = np.load(open('valid_labels.npy','rb'))

In [13]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_valid = X_valid.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
X_valid = X_valid / 255.0

In [8]:
# one hot encode outputs
#y_train = keras.utils.to_categorical(y_train)
#y_test = keras.utils.to_categorical(y_test)
#y_valid = keras.utils.to_categorical(y_valid)
#num_classes = y_train.shape[1]
#print(num_classes)

y_valid = preprocessing.minmax_scale(y_valid, feature_range=(0,119))
y_train = preprocessing.minmax_scale(y_train, feature_range=(0,119))
y_test = preprocessing.minmax_scale(y_test, feature_range=(0,119))

120


In [17]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(56,56,3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))
# Compile model
epochs = 1
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 56, 3)         16160     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 56, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 56, 3)         9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 28, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 896)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               459264    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

In [18]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 16464 samples, validate on 2058 samples
Epoch 1/1
16464/16464 [==============================] - 364s 22ms/step - loss: 4.4786 - acc: 0.0000e+00 - val_loss: 4.3568 - val_acc: 0.0000e+00
Accuracy: 0.00%


In [20]:
print(y_valid)

[ 100.94999679   45.33151231   21.08638087 ...,   74.73160229   58.83842149
   97.55032457]
